<a href="https://colab.research.google.com/github/haimin777/aramco_2019/blob/master/ARAMCO_probabilistic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:

import os

In [0]:
os.chdir('/content/drive/My Drive/AIgym/probability')

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import warnings
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline


from PIL import Image
import cv2

from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

# Dependency imports
from absl import flags
import matplotlib
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp

from tf_probability.examples.models.bayesian_resnet import bayesian_resnet
from tf_probability.examples.models.bayesian_vgg import bayesian_vgg

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

matplotlib.use("Agg")
from matplotlib import figure  
from matplotlib.backends import backend_agg
warnings.simplefilter(action="ignore")
tfd = tfp.distributions

import seaborn as sns  
HAS_SEABORN = True

IMAGE_SHAPE = [128, 128, 1]

In [0]:
# Get and resize train images based on avilable masks
def get_data(path, train=True, num_classes=10, img_size=128):
    ids = next(os.walk(path + "images"))[2]
    X = np.zeros((len(ids), img_size, img_size, 1), dtype=np.float32)
    if train:
        #y = np.zeros((len(ids), img_size, img_size, num_classes), dtype=np.float32)
        y = []
    print('Getting and resizing images ... ')
    for n, id_ in tqdm_notebook(enumerate(ids), total=len(ids)):
     
        # Load images
        img = load_img(path + 'images/' + id_, color_mode = "grayscale")
        x_img = img_to_array(img)
        x_img = cv2.equalizeHist(x_img.astype(np.uint8))
        x_img = resize(x_img, (img_size, img_size, 1), mode='constant', preserve_range=True)

        # calculate label based on mask
        if train:
            mask = np.amax(np.array(Image.open(path + 'masks/' + id_.split('_')[0]+'_xlinescustom.png')))
            #mask = resize(mask, (128, 128, num_classes), mode='constant', preserve_range=True)
            

        # Save images
        X[n, ..., 0] = x_img.squeeze() / 255
        if train:
            y.append([int(mask/255)])
    print('Done!')
    if train:
        return X, np.array(y)
    else:
        return X
    


In [9]:
X, y = get_data('/content/drive/My Drive/datasets/trap_seismic_dataset/', 128)

Getting and resizing images ... 



Done!


In [0]:
FLAGS = {"learning_rate": 0.0001,
         "epochs": 1000,
         "batch_size": 10,
         "data_dir": os.path.join('/content/drive/My Drive/AIgym/probability',
                                         "bayesian_neural_network/data"),

         "model_dir": os.path.join('/content/drive/My Drive/AIgym/probability',
                         "bayesian_neural_network/"),
         "eval_freq": 400,
         "num_monte_carlo": 50, #help Network draws to compute predictive probabilities.
         "architecture": "resnet",
         "kernel_posterior_scale_mean": -9.0, # help="Initial kernel posterior mean of the scale (log var) for q(w)")
         "kernel_posterior_scale_constraint": 0.2, #help="Posterior kernel constraint for the scale (log var) of q(w).")
         "kl_annealing": 50, #help="Epochs to anneal the KL term (anneals from 0 to 1)")
         "subtract_pixel_mean": True, #help="Boolean for normalizing the images")
         "fake_data": None, #help="If true, uses fake data. Defaults to real data.")
         
          }




In [0]:
#visualicing and metrics


def plot_weight_posteriors(names, qm_vals, qs_vals, fname):
  """Save a PNG plot with histograms of weight means and stddevs.
  Args:
    names: A Python `iterable` of `str` variable names.
    qm_vals: A Python `iterable`, the same length as `names`,
      whose elements are Numpy `array`s, of any shape, containing
      posterior means of weight varibles.
    qs_vals: A Python `iterable`, the same length as `names`,
      whose elements are Numpy `array`s, of any shape, containing
      posterior standard deviations of weight varibles.
    fname: Python `str` filename to save the plot to.
  """
  fig = figure.Figure(figsize=(10, 20))
  canvas = backend_agg.FigureCanvasAgg(fig)

  ax = fig.add_subplot(1, 2, 1)
  for n, qm in zip(names, qm_vals):
    sns.distplot(qm.flatten(), ax=ax, label=n)
  ax.set_title("weight means")
  ax.set_xlim([-1.5, 1.5])
  ax.legend()

  ax = fig.add_subplot(1, 2, 2)
  for n, qs in zip(names, qs_vals):
    sns.distplot(qs.flatten(), ax=ax)
  ax.set_title("weight stddevs")
  ax.set_xlim([0, 1.])

  fig.tight_layout()
  canvas.print_figure(fname, format="png")
  print("saved {}".format(fname))


def plot_heldout_prediction(input_vals, probs,
                            fname, n=2, title=""):
  """Save a PNG plot visualizing posterior uncertainty on heldout data.
  Args:
    input_vals: A `float`-like Numpy `array` of shape
      `[num_heldout] + IMAGE_SHAPE`, containing heldout input images.
    probs: A `float`-like Numpy array of shape `[num_monte_carlo,
      num_heldout, num_classes]` containing Monte Carlo samples of
      class probabilities for each heldout sample.
    fname: Python `str` filename to save the plot to.
    n: Python `int` number of datapoints to vizualize.
    title: Python `str` title for the plot.
  """
  fig = figure.Figure(figsize=(10, 10*n))
  canvas = backend_agg.FigureCanvasAgg(fig)
  for i in range(n):
    ax = fig.add_subplot(n, 3, 3*i + 1)
    ax.imshow(input_vals[i, :,:,0].reshape(IMAGE_SHAPE[:-1]), interpolation="None")

    ax = fig.add_subplot(n, 3, 3*i + 2)
    for prob_sample in probs:
      sns.barplot(np.arange(2), prob_sample[i, :], alpha=0.1, ax=ax) # was 10
      ax.set_ylim([0, 1])
    ax.set_title("posterior samples")

    ax = fig.add_subplot(n, 3, 3*i + 3)
    sns.barplot(np.arange(2), np.mean(probs[:, i, :], axis=0), ax=ax) # was 10
    ax.set_ylim([0, 1])
    ax.set_title("predictive probs")
  fig.suptitle(title)
  fig.tight_layout()

  canvas.print_figure(fname, format="png")
  print("saved {}".format(fname))


def build_input_pipeline(x_train, x_test, y_train, y_test,
                         batch_size, valid_size):
  """Build an Iterator switching between train and heldout data."""

  x_train = x_train.astype("float32")
  x_test = x_test.astype("float32")

  x_train /= 255
  x_test /= 255

  y_train = y_train.flatten()
  y_test = y_test.flatten()

  if FLAGS['subtract_pixel_mean']:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

  print("x_train shape:" + str(x_train.shape))
  print(str(x_train.shape[0]) + " train samples")
  print(str(x_test.shape[0]) + " test samples")

  # Build an iterator over training batches.
  training_dataset = tf.data.Dataset.from_tensor_slices(
      (x_train, np.int32(y_train)))
  training_batches = training_dataset.shuffle(
      50000, reshuffle_each_iteration=True).repeat().batch(batch_size)
  training_iterator = tf.compat.v1.data.make_one_shot_iterator(training_batches)

  # Build a iterator over the heldout set with batch_size=heldout_size,
  # i.e., return the entire heldout set as a constant.
  heldout_dataset = tf.data.Dataset.from_tensor_slices(
      (x_test, np.int32(y_test)))
  heldout_batches = heldout_dataset.repeat().batch(valid_size)
  heldout_iterator = tf.compat.v1.data.make_one_shot_iterator(heldout_batches)

  # Combine these into a feedable iterator that can switch between training
  # and validation inputs.
  handle = tf.compat.v1.placeholder(tf.string, shape=[])
  feedable_iterator = tf.compat.v1.data.Iterator.from_string_handle(
      handle, training_batches.output_types, training_batches.output_shapes)
  images, labels = feedable_iterator.get_next()

  return images, labels, handle, training_iterator, heldout_iterator

In [13]:

#load data
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=2018)

(images, labels, handle, training_iterator, heldout_iterator) = build_input_pipeline(x_train, x_test, y_train, y_test,
                                            FLAGS['batch_size'], 500)

x_train shape:(135, 128, 128, 1)
135 train samples
24 test samples
Instructions for updating:
Use `tf.compat.v1.data.get_output_types(dataset)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(dataset)`.


In [14]:
model_fn = bayesian_resnet
model = model_fn(
      IMAGE_SHAPE,
      num_classes=2,
      kernel_posterior_scale_mean=FLAGS['kernel_posterior_scale_mean'],
      kernel_posterior_scale_constraint=FLAGS['kernel_posterior_scale_constraint'])
logits = model(images)
labels_distribution = tfd.Categorical(logits=logits)

  # Perform KL annealing. The optimal number of annealing steps
  # depends on the dataset and architecture.
t = tf.compat.v2.Variable(0.0)
kl_regularizer = t / (FLAGS['kl_annealing'] * len(x_train) / FLAGS['batch_size'])

  # Compute the -ELBO as the loss. The kl term is annealed from 0 to 1 over
  # the epochs specified by the kl_annealing flag.
log_likelihood = labels_distribution.log_prob(labels)
neg_log_likelihood = -tf.reduce_mean(input_tensor=log_likelihood)
kl = sum(model.losses) / len(x_train) * tf.minimum(1.0, kl_regularizer)
loss = neg_log_likelihood + kl

Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [15]:
  # Build metrics for evaluation. Predictions are formed from a single forward
  # pass of the probabilistic layers. They are cheap but noisy
  # predictions.
  names = []
  qmeans = []
  qstds = []
  for i, layer in enumerate(model.layers):
    try:
      q = layer.kernel_posterior
    except AttributeError:
      continue
    names.append("Layer {}".format(i))
    qmeans.append(q.mean())
    qstds.append(q.stddev())

  predictions = tf.argmax(input=logits, axis=1)
  with tf.compat.v1.name_scope("train"):
    train_accuracy, train_accuracy_update_op = tf.compat.v1.metrics.accuracy(
        labels=labels, predictions=predictions)
    opt = tf.compat.v1.train.AdamOptimizer(FLAGS['learning_rate'])
    train_op = opt.minimize(loss)
    update_step_op = tf.compat.v1.assign(t, t + 1)

  with tf.compat.v1.name_scope("valid"):
    valid_accuracy, valid_accuracy_update_op = tf.compat.v1.metrics.accuracy(
        labels=labels, predictions=predictions)

  init_op = tf.group(tf.compat.v1.global_variables_initializer(),
                     tf.compat.v1.local_variables_initializer())

  stream_vars_valid = [
      v for v in tf.compat.v1.local_variables() if "valid/" in v.name
  ]
  reset_valid_op = tf.compat.v1.variables_initializer(stream_vars_valid)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
  
  with tf.compat.v1.Session() as sess:
    sess.run(init_op)
    saver = tf.train.Saver()

    # Run the training loop
    train_handle = sess.run(training_iterator.string_handle())
    heldout_handle = sess.run(heldout_iterator.string_handle())
    training_steps = int(
        round(FLAGS['epochs'] * (len(x_train) / FLAGS['batch_size'])))
    for step in range(training_steps):
      _ = sess.run([train_op,
                    train_accuracy_update_op,
                    update_step_op],
                   feed_dict={handle: train_handle})

      # Manually print the frequency
      if step % 100 == 0:
        loss_value, accuracy_value, kl_value = sess.run(
            [loss, train_accuracy, kl], feed_dict={handle: train_handle})
        print(
            "Step: {:>3d} Loss: {:.3f} Accuracy: {:.3f} KL: {:.3f}".format(
                step, loss_value, accuracy_value, kl_value))

      if (step + 1) % FLAGS['eval_freq'] == 0:
        # Compute log prob of heldout set by averaging draws from the model:
        # p(heldout | train) = int_model p(heldout|model) p(model|train)
        #                   ~= 1/n * sum_{i=1}^n p(heldout | model_i)
        # where model_i is a draw from the posterior
        # p(model|train).
        probs = np.asarray([sess.run((labels_distribution.probs),
                                     feed_dict={handle: heldout_handle})
                            for _ in range(FLAGS['num_monte_carlo'])])
        mean_probs = np.mean(probs, axis=0)

        image_vals, label_vals = sess.run(
            (images, labels), feed_dict={handle: heldout_handle})
        heldout_lp = np.mean(np.log(mean_probs[np.arange(mean_probs.shape[0]),
                                               label_vals.flatten()]))
        print(" ... Held-out nats: {:.3f}".format(heldout_lp))

        # Calculate validation accuracy
        for _ in range(20):
          sess.run(
              valid_accuracy_update_op, feed_dict={handle: heldout_handle})
        valid_value = sess.run(
            valid_accuracy, feed_dict={handle: heldout_handle})

        print(
            " ... Validation Accuracy: {:.3f}".format(valid_value))
        
        qm_vals, qs_vals = sess.run((qmeans, qstds))
        saver.save(sess, 'my_bayesian_model')

        if HAS_SEABORN:
          plot_weight_posteriors(names, qm_vals, qs_vals,
                                 fname=os.path.join(
                                     FLAGS['model_dir'],
                                     "step{:05d}_weights_equHist.png".format(step)))

          plot_heldout_prediction(image_vals, probs,
                                  fname=os.path.join(
                                      FLAGS['model_dir'],
                                      "step{:05d}_pred_equHist.png".format(step)),
                                  title="mean heldout logprob {:.2f}"
                                  .format(heldout_lp))

        sess.run(reset_valid_op)

Step:   0 Loss: 1510.061 Accuracy: 0.500 KL: 472.952
Step: 100 Loss: 47752.605 Accuracy: 0.546 KL: 47703.770
Step: 200 Loss: 94884.477 Accuracy: 0.555 KL: 94765.672
Step: 300 Loss: 141647.359 Accuracy: 0.547 KL: 141647.359
